# Find ppl

## Download data (for colab)

In [ ]:
# from google.colab import auth
# auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip
!rm annotations_trainval2017.zip
!cp -r /content/drive/My\ Drive/scripts ./scripts
!python3 -m pip install aiofiles aiohttp pycocotools aiohttp[speedups] yarl==1.2.6


In [ ]:
!python3 ./scripts/async_gather_data.py \
    --anns_path ./annotations/instances_train2017.json \
    --img_dst ./images/train \
    --mask_dst ./masks/train
!python3 ./scripts/async_gather_data.py \
    --anns_path ./annotations/instances_val2017.json \
    --img_dst ./images/val \
    --mask_dst ./masks/val


In [ ]:
!rm ./images/train/000000123473.jpg
!rm ./masks/train/000000123473.png


In [ ]:
from pathlib import Path

train_dir = Path('./images/train')
max_images = 1000

for src_image in sorted(train_dir.iterdir())[:max_images]:
    src_mask = Path('./masks/train') / Path(src_image.stem + '.png')
    dst_mask = Path('./masks/val') / Path(src_image.stem + '.png')
    dst_img = Path('./images/val') / Path(src_image.name)
    src_image.replace(dst_img)
    src_mask.replace(dst_mask)
    

## Imports and initial env settings

In [ ]:
import os
from functools import partial
from pathlib import Path

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 1.x
except Exception:
    pass
  
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [ ]:
tf.enable_v2_behavior()


In [ ]:
DIRECTORY = './'
IMAGES_PATH = './images/'
MASKS_PATH = './masks/'
TRAIN_SIZE = 10996
VAL_SIZE = 1494
BATCH_SIZE = 4
IMG_SIZE = (512, 512)
SEED = 111
EPOCHS_FOR_FULL_TRAIN_PASS = 1

np.random.seed(SEED)
try:
    tf.random.set_seed(SEED)
except AttributeError:  # if tensorflow == 1.x
    tf.random.set_random_seed(SEED)
    

## Making generators for feeding models with data

In [ ]:
# we create two instances with the same arguments
data_gen_args = dict(
    # width_shift_range=0.1,
    # height_shift_range=0.1,
    rotation_range=10,
    # fill_mode='wrap',
    zoom_range=[0.9, 1],
    horizontal_flip=True,
    vertical_flip=True,
)
image_datagen = keras.preprocessing.image.ImageDataGenerator(
    **data_gen_args, rescale=1/255)
mask_datagen = keras.preprocessing.image.ImageDataGenerator(
    **data_gen_args, dtype=np.uint8)
val_image_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
val_mask_datagen = keras.preprocessing.image.ImageDataGenerator(dtype=np.uint8)

# Provide the same seed and keyword arguments to the fit and flow methods
flow_args = dict(
    target_size=IMG_SIZE,
    class_mode=None,
    batch_size=BATCH_SIZE,
    seed=SEED
)

train_image_generator = image_datagen.flow_from_directory(
    IMAGES_PATH,
    **flow_args,
    classes=['train'],
    color_mode='rgb',
)

train_mask_generator = mask_datagen.flow_from_directory(
    MASKS_PATH,
    **flow_args,
    classes=['train'],
    color_mode='grayscale',
)

val_image_generator = val_image_datagen.flow_from_directory(
    IMAGES_PATH,
    **flow_args,
    classes=['val'],
    color_mode='rgb',
)

val_mask_generator = val_mask_datagen.flow_from_directory(
    MASKS_PATH,
    **flow_args,
    classes=['val'],
    color_mode='grayscale',
)

# combine generators into one which yields image and masks
def generator(images, masks):
    while True:
        yield (next(images), next(masks))

train_generator = generator(train_image_generator, train_mask_generator)
val_generator = generator(val_image_generator, val_mask_generator)


In [ ]:
batch = next(val_generator)
for img, mask in zip(batch[0], batch[1]):
    plt.imshow(img)
    plt.imshow(mask.reshape(IMG_SIZE), alpha=0.3)
    plt.axis('off')
    plt.show()


## U-net model

### Some partial functions to enter less code

In [ ]:
DefaultMaxPool2D = partial(keras.layers.MaxPooling2D,
                           padding='valid', pool_size=(2, 2), strides=(2, 2))
DefaultConv2DTranspose = partial(keras.layers.Conv2DTranspose,
                                 padding='valid',
                                 kernel_size=(2, 2), strides=(2, 2))
DefaultUpSampling2D = partial(keras.layers.UpSampling2D,
                              size=(2, 2), data_format='channels_last',
                              interpolation='bilinear')


### Main building functions for u-net model and it's layers

In [ ]:
def conv2d_block(inputs, filters, kernel_size=(3, 3), n_layers=2, alpha=0.2,
                 use_batch_norm=True, padding='same'):
    for i in range(n_layers):
        inputs = keras.layers.Conv2D(
            filters, kernel_size, padding=padding)(inputs)
        inputs = keras.layers.LeakyReLU(alpha=alpha)(inputs)
        if use_batch_norm:
            inputs = keras.layers.BatchNormalization()(inputs)
    return inputs

def upsample_block(inputs, filters, concat, kernel_size=(2, 2), strides=(2, 2),
                   alpha=0.3, padding='valid', use_transposed_conv=False,
                   conv_params=None):
    if conv_params is None:
        conv_params = {'filters': 2 * filters}
    conv_layer = conv2d_block(inputs, **conv_params)
    if use_transposed_conv:
        up_layer = DefaultConv2DTranspose(
            filters=filters, kernel_size=kernel_size)(conv_layer)
    else:
        up_layer = DefaultUpSampling2D(size=kernel_size)(conv_layer)
        up_layer = keras.layers.Conv2D(
            filters=filters, kernel_size=kernel_size, padding='same')(up_layer)
    activation = keras.layers.LeakyReLU(alpha=alpha)(up_layer)
    concat_layer = keras.layers.Concatenate(axis=-1)([concat, activation])
    return concat_layer

def downsample_block(inputs, filters, conv_params=None):
    if conv_params is None:
        conv_params = {}
    conv_layer = conv2d_block(inputs, filters, **conv_params)
    maxpool_layer = DefaultMaxPool2D()(conv_layer)
    return conv_layer, maxpool_layer

def unet(input_shape=None, classes=2, use_sigmoid=True):
    inputs = keras.layers.Input(shape=input_shape)

    # downsampling stack
    conv1, down1 = downsample_block(inputs, 64)
    conv2, down2 = downsample_block(down1, 128)
    conv3, down3 = downsample_block(down2, 256)
    conv4, down4 = downsample_block(down3, 512)

    # upsampling stack
    up1 = upsample_block(down4, 512, conv4)
    up2 = upsample_block(up1, 256, conv3)
    up3 = upsample_block(up2, 128, conv2)
    up4 = upsample_block(up3, 64, conv1)
    
    conv_block_last = conv2d_block(up4, 64)
    if classes == 2 and use_sigmoid:
        outputs = keras.layers.Conv2D(
            1, (1, 1), activation='sigmoid')(conv_block_last)
    else:
        outputs = keras.layers.Conv2D(
            classes, (1, 1), activation='softmax')(conv_block_last)

    model = keras.models.Model(inputs=inputs, outputs=outputs)
    return model


In [ ]:
m = tf.keras.metrics.MeanIoU(num_classes=2)
def mean_iou(y_true, y_pred):
    metric = m(y_true, tf.round(y_pred))
    m.reset_states()
    return metric


def bce_dc_loss(y_true, y_pred):
    """ Binary crossentropy + Dice Coef loss. """

    def dice_coef(y_true, y_pred, smooth=1):
        """ Dice coef implementation.
        
        """
        numerator = 2 * keras.backend.sum(keras.backend.abs(y_true * y_pred), axis=-1)
        denominator = (keras.backend.sum(keras.backend.square(y_true), -1) + keras.backend.sum(keras.backend.square(y_pred), -1))
        return (numerator + smooth) / (denominator + smooth)

    def dice_coef_loss(y_true, y_pred):
        return 1 - dice_coef(y_true, y_pred)

    return (keras.losses.binary_crossentropy(y_true, y_pred) +
            dice_coef_loss(y_true, y_pred))
    

In [ ]:
model = unet(input_shape=(None, None, 3))

# model = keras.models.load_model(
#     './drive/My Drive/model_checkpoints/unet_015.hdf5', 
#     custom_objects={
#         'bce_dc_loss': bce_dc_loss,
#         'mean_iou': mean_iou,
#     },
# )

model.compile(
   optimizer=keras.optimizers.Adam(1e-3),
   loss=bce_dc_loss,
#    loss=keras.losses.binary_crossentropy,
   metrics=[mean_iou],
)

model.summary()
keras.utils.plot_model(model, show_shapes=True, to_file='unet.png')


In [ ]:
preds = model.predict(batch[0])
print(mean_iou(batch[1], preds))
for img, mask, pred in zip(batch[0], batch[1], preds):
    fg, (ax1, ax2, ax3, ax4) = plt.subplots(nrows=1, ncols=4, figsize=(16, 4))
    ax1.imshow(img)
    ax1.set_title('Input image')
    ax1.axis('off')

    ax2.imshow(pred.reshape(IMG_SIZE), cmap='inferno')
    ax2.set_title('Predicted raw values')
    ax2.axis('off')

    ax3.imshow(img)
    ax3.imshow(pred.reshape(IMG_SIZE).round(), alpha=0.6, cmap='inferno')
    ax3.set_title('Predicted mask (rounded raw values)')
    ax3.axis('off')

    ax4.imshow(img)
    ax4.imshow(mask.reshape(IMG_SIZE), alpha=0.6, cmap='inferno')
    ax4.axis('off')
    ax4.set_title('True mask')

    plt.show()
    

## Training the model

### Callbacks for model saving, scheduling learning rate, early stopping and logging to csv file.

In [ ]:
csv_name = 'log_unet.csv'
if os.path.exists('/content/drive/'):
    checkpoint_dir_path = os.path.join(
        '/content/drive/My Drive/', 'model_checkpoints/')
    csv_log_path = os.path.join('/content/drive/My Drive/', csv_name)
else:
    checkpoint_dir_path = os.path.join(
        DIRECTORY, 'model_checkpoints/')
    csv_log_path = os.path.join(DIRECTORY, csv_name)
    

checkpoint_name ='unet_{epoch:03d}.hdf5'
checkpoint_path = os.path.join(checkpoint_dir_path, checkpoint_name)

try:
    os.makedirs(checkpoint_dir_path)
except FileExistsError:
    print('Exists:', checkpoint_dir_path)


checkpoint = keras.callbacks.ModelCheckpoint(
    checkpoint_path, mode='auto',
    # save_freq=(TRAIN_SIZE//BATCH_SIZE//EPOCHS_FOR_FULL_TRAIN_PASS)*BATCH_SIZE*EPOCHS_FOR_FULL_TRAIN_PASS,
    # period=4,
    verbose=0
    )

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', mode='min', factor=0.2,
    patience=EPOCHS_FOR_FULL_TRAIN_PASS+1, min_lr=1e-7)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=EPOCHS_FOR_FULL_TRAIN_PASS+3,
    restore_best_weights=True)

csv_logger = keras.callbacks.CSVLogger(
    csv_log_path, separator=',', append=True)

callbacks = [
    reduce_lr,
    early_stopping,
    csv_logger,
    checkpoint,
]


In [ ]:
def skip_epochs(generator, epochs=1, steps_per_epoch=None):
    if steps_per_epoch is None:
        steps_per_epoch = TRAIN_SIZE//BATCH_SIZE//EPOCHS_FOR_FULL_TRAIN_PASS
    from tqdm import tqdm
    for i in tqdm(range(epochs * steps_per_epoch)):
        next(generator)
        

### So let the training begin!

In [ ]:
try:
    train_history = model.fit_generator(
        generator=train_generator,
        validation_data=val_generator,
        epochs=20,
        steps_per_epoch=TRAIN_SIZE//BATCH_SIZE//EPOCHS_FOR_FULL_TRAIN_PASS,
        validation_steps=VAL_SIZE//BATCH_SIZE,
        callbacks=callbacks,
        initial_epoch=0,
    )
except Exception as e:
    import traceback
    with open(DIRECTORY + 'error.txt', 'w') as file:
        file.write('While training the network exception occured:')
        file.write(traceback.format_exc())
    raise e
    

### And now let's see what we've got after the training.

In [ ]:
n_iter = 4
i = 0
for _ in range(n_iter):
    preds = model.predict(batch[0])
    print(mean_iou(batch[1], preds))
    for img, mask, pred in zip(batch[0], batch[1], preds):
        
        fg, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(nrows=1, ncols=5, figsize=(20, 5))
        ax1.imshow(img)
        ax1.set_title('Input image')
        ax1.axis('off')

        ax2.imshow(pred.reshape(IMG_SIZE), cmap='inferno')
        ax2.set_title('Predicted raw values')
        ax2.axis('off')

        ax3.imshow(img)
        ax3.imshow(pred.reshape(IMG_SIZE).round(), alpha=0.6, cmap='inferno')
        ax3.set_title('Predicted mask (rounded raw values)')
        ax3.axis('off')


        ax4.imshow(img)
        ax4.imshow((pred.reshape(IMG_SIZE) > 0.25).astype(int), alpha=0.6, cmap='inferno')
        ax4.set_title('Predicted mask (> 0.25)')
        ax4.axis('off')

        ax5.imshow(img)
        ax5.imshow(mask.reshape(IMG_SIZE), alpha=0.6, cmap='inferno')
        ax5.axis('off')
        ax5.set_title('True mask')

        plt.savefig(f'./batches/batch_visualisation_{i}.png')
        i+=1
        plt.show()
        
        batch = next(val_generator)
    